# Module 10: Data Cleaning and Preprocessing

## Topics Covered
1. Common Data Quality Issues
2. Handling Duplicates
3. Dealing with Missing Values (Strategies)
4. Outlier Detection and Treatment
5. Data Normalization and Standardization
6. Encoding Categorical Variables
7. Feature Engineering Basics
8. Data Cleaning Pipeline Project

## Learning Objectives

By the end of this module, you will be able to:
- Identify and diagnose common data quality issues
- Handle duplicates and missing values appropriately
- Detect and treat outliers using various methods
- Scale and normalize numeric features
- Encode categorical variables for machine learning
- Create new features from existing data
- Build reusable data cleaning pipelines

---

---
# Section 1: Common Data Quality Issues
---

## Why Data Cleaning Matters

Real-world data is messy. Data scientists spend 60-80% of their time on data cleaning and preparation. Common issues include:

- **Missing values**: Empty cells, NaN, None
- **Duplicates**: Repeated records
- **Inconsistent formatting**: "USA", "U.S.A.", "United States"
- **Invalid data**: Negative ages, future dates
- **Outliers**: Extreme values that may be errors
- **Wrong data types**: Numbers stored as strings

### The Cost of Dirty Data

- Incorrect analysis and conclusions
- Poor model performance
- Wasted computational resources
- Loss of stakeholder trust

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
%matplotlib inline
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

print("Libraries imported successfully")

In [ ]:
# Create a messy dataset to demonstrate cleaning techniques

np.random.seed(42)

messy_data = pd.DataFrame({
    'customer_id': [1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'name': ['John Smith', 'jane doe', 'ROBERT BROWN', 'Robert Brown', 'Alice Wilson',
             'Bob Johnson', None, 'Carol White', 'David Lee', 'Eva Martinez',
             'Frank Miller', 'Grace Kim', 'Henry Chen', 'Ivy Taylor', 'Jack Davis', 'Kate Moore'],
    'age': [25, 30, -5, 35, 150, 28, 42, 33, None, 29, 45, 38, 27, 31, 1000, 26],
    'email': ['john@email.com', 'jane@email.com', 'robert@email.com', 'robert@email.com',
              'invalid-email', 'bob@email.com', 'carol@email.com', 'carol@email.com',
              'david@email.com', None, 'frank@email.com', 'grace@email.com',
              'henry@email.com', 'ivy@email.com', 'jack@email.com', 'kate@email.com'],
    'income': [50000, 60000, 75000, 75000, 45000, None, 80000, 55000, 
               62000, 58000, 1000000, 72000, 48000, 65000, 52000, 69000],
    'city': ['New York', 'new york', 'NYC', 'New York', 'Los Angeles',
             'LA', 'Chicago', 'chicago', 'Houston', 'Phoenix',
             'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose', 'Austin'],
    'signup_date': ['2023-01-15', '2023/02/20', 'March 5, 2023', '2023-03-05',
                    '2023-04-10', '2023-05-15', '2023-06-20', '15-07-2023',
                    '2023-08-25', '2023-09-30', '2023-10-05', '2023-11-10',
                    '2023-12-15', '2024-01-20', '2024-02-25', '2024-03-01']
})

print("Messy Dataset Created:")
print(messy_data)

In [ ]:
# Example: Diagnosing data quality issues

def diagnose_data_quality(df):
    """Comprehensive data quality diagnosis."""
    print("DATA QUALITY REPORT")
    print("=" * 60)
    
    # Basic info
    print(f"\n1. BASIC INFO")
    print(f"   Rows: {len(df)}, Columns: {len(df.columns)}")
    
    # Missing values
    print(f"\n2. MISSING VALUES")
    missing = df.isnull().sum()
    if missing.sum() > 0:
        for col, count in missing[missing > 0].items():
            print(f"   {col}: {count} ({count/len(df)*100:.1f}%)")
    else:
        print("   None")
    
    # Duplicates
    print(f"\n3. DUPLICATES")
    print(f"   Full row duplicates: {df.duplicated().sum()}")
    
    # Data types
    print(f"\n4. DATA TYPES")
    for col, dtype in df.dtypes.items():
        print(f"   {col}: {dtype}")
    
    # Numeric column stats
    print(f"\n5. NUMERIC COLUMNS - POTENTIAL ISSUES")
    numeric_cols = df.select_dtypes(include=np.number).columns
    for col in numeric_cols:
        min_val, max_val = df[col].min(), df[col].max()
        print(f"   {col}: min={min_val}, max={max_val}")
    
    print("\n" + "=" * 60)

diagnose_data_quality(messy_data)

---
# Section 2: Handling Duplicates
---

Duplicates can occur from data entry errors, system glitches, or merging datasets.

In [ ]:
# Example: Identifying duplicates

# Check for exact duplicates (all columns match)
exact_duplicates = messy_data[messy_data.duplicated(keep=False)]
print(f"Exact duplicate rows: {len(exact_duplicates) // 2}")
print(exact_duplicates)

In [ ]:
# Example: Check duplicates based on specific columns

# Duplicates based on customer_id
id_duplicates = messy_data[messy_data.duplicated(subset=['customer_id'], keep=False)]
print("Duplicate customer IDs:")
print(id_duplicates)

# Duplicates based on email
email_duplicates = messy_data[messy_data.duplicated(subset=['email'], keep=False)].dropna(subset=['email'])
print("\nDuplicate emails:")
print(email_duplicates)

In [ ]:
# Example: Removing duplicates

df = messy_data.copy()
print(f"Before: {len(df)} rows")

# Remove exact duplicates (keep first occurrence)
df_no_dups = df.drop_duplicates()
print(f"After removing exact duplicates: {len(df_no_dups)} rows")

# Remove duplicates based on customer_id (keep first)
df_unique_id = df.drop_duplicates(subset=['customer_id'], keep='first')
print(f"After removing duplicate IDs: {len(df_unique_id)} rows")

In [ ]:
# Example: Different strategies for handling duplicates

df = messy_data.copy()

# Keep first occurrence
keep_first = df.drop_duplicates(subset=['customer_id'], keep='first')

# Keep last occurrence
keep_last = df.drop_duplicates(subset=['customer_id'], keep='last')

# Remove all duplicates (keep none)
keep_none = df.drop_duplicates(subset=['customer_id'], keep=False)

print(f"Original: {len(df)} rows")
print(f"Keep first: {len(keep_first)} rows")
print(f"Keep last: {len(keep_last)} rows")
print(f"Keep none: {len(keep_none)} rows")

---
# Section 3: Dealing with Missing Values
---

Missing values require careful consideration. The right strategy depends on:
- How much data is missing
- Why data is missing (random vs. systematic)
- The importance of the variable

In [ ]:
# Load real data with missing values
sales = pd.read_csv('assets/datasets/sales_data.csv')
employees = pd.read_csv('assets/datasets/employees.csv')

# Check missing values
print("Missing values in Sales:")
print(sales.isnull().sum()[sales.isnull().sum() > 0])

print("\nMissing values in Employees:")
print(employees.isnull().sum()[employees.isnull().sum() > 0])

In [ ]:
# Example: Visualizing missing values

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Missing value counts
sales_missing = sales.isnull().sum()
sales_missing = sales_missing[sales_missing > 0]
axes[0].bar(sales_missing.index, sales_missing.values, color='coral')
axes[0].set_title('Missing Values in Sales Data')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Missing value pattern (heatmap)
# Sample for visualization
sample = sales.sample(100, random_state=42)
sns.heatmap(sample.isnull(), cbar=True, yticklabels=False, ax=axes[1])
axes[1].set_title('Missing Value Pattern (Sample)')

plt.tight_layout()
plt.show()

## Missing Value Strategies

| Strategy | When to Use |
|----------|-------------|
| **Delete rows** | Small % missing, data is MCAR |
| **Delete columns** | >50% missing, not important |
| **Fill with mean/median** | Numeric, random missing |
| **Fill with mode** | Categorical data |
| **Fill with constant** | Known default value |
| **Forward/backward fill** | Time series data |
| **Interpolation** | Ordered numeric data |
| **Model-based imputation** | Complex patterns |

In [ ]:
# Example: Strategy 1 - Delete rows with missing values

df = sales.copy()
print(f"Before: {len(df)} rows")

# Drop rows with ANY missing value
df_dropped_any = df.dropna()
print(f"After dropna(): {len(df_dropped_any)} rows")

# Drop rows only if specific columns are missing
df_dropped_subset = df.dropna(subset=['unit_price', 'sales_rep'])
print(f"After dropna(subset): {len(df_dropped_subset)} rows")

In [ ]:
# Example: Strategy 2 - Fill with statistics

df = sales.copy()

# Fill numeric with mean
mean_price = df['unit_price'].mean()
df['unit_price_mean'] = df['unit_price'].fillna(mean_price)

# Fill numeric with median (better for skewed data)
median_price = df['unit_price'].median()
df['unit_price_median'] = df['unit_price'].fillna(median_price)

# Fill categorical with mode
mode_rep = df['sales_rep'].mode()[0]
df['sales_rep_filled'] = df['sales_rep'].fillna(mode_rep)

print(f"Mean price: ${mean_price:.2f}")
print(f"Median price: ${median_price:.2f}")
print(f"Mode sales rep: {mode_rep}")

# Verify
print(f"\nRemaining missing in unit_price_mean: {df['unit_price_mean'].isnull().sum()}")
print(f"Remaining missing in sales_rep_filled: {df['sales_rep_filled'].isnull().sum()}")

In [ ]:
# Example: Strategy 3 - Group-based imputation

df = sales.copy()

# Fill missing unit_price with category median
df['unit_price_filled'] = df.groupby('category')['unit_price'].transform(
    lambda x: x.fillna(x.median())
)

# Check result
print("Category-based median prices:")
print(df.groupby('category')['unit_price'].median())
print(f"\nRemaining missing: {df['unit_price_filled'].isnull().sum()}")

In [ ]:
# Example: Strategy 4 - Fill with constant/indicator

df = sales.copy()

# Fill with a constant value
df['sales_rep_const'] = df['sales_rep'].fillna('Unknown')

# Create a missing indicator (useful for ML)
df['rating_missing'] = df['customer_rating'].isnull().astype(int)
df['customer_rating_filled'] = df['customer_rating'].fillna(0)

print("Sales rep values after filling:")
print(df['sales_rep_const'].value_counts())

print(f"\nMissing rating indicator sum: {df['rating_missing'].sum()}")

In [ ]:
# Example: Strategy 5 - Forward/backward fill (for time series)

# Create time series example
ts_data = pd.DataFrame({
    'date': pd.date_range('2023-01-01', periods=10),
    'value': [100, np.nan, np.nan, 110, 115, np.nan, 120, 125, np.nan, 130]
})

print("Original:")
print(ts_data)

# Forward fill
ts_data['ffill'] = ts_data['value'].ffill()

# Backward fill
ts_data['bfill'] = ts_data['value'].bfill()

# Interpolation
ts_data['interpolate'] = ts_data['value'].interpolate()

print("\nAfter filling:")
print(ts_data)

## Practice Exercise 3.1

**Task:** Clean the employees dataset:
1. Check for missing values in all columns
2. Fill missing performance_rating with the department median
3. Fill missing email with a pattern based on name
4. Handle missing bonus appropriately

In [ ]:
# Your code here


In [ ]:
# Solution 3.1

employees = pd.read_csv('assets/datasets/employees.csv')

# 1. Check missing values
print("1. Missing Values:")
print(employees.isnull().sum())

# 2. Fill performance_rating with department median
employees['performance_rating'] = employees.groupby('department')['performance_rating'].transform(
    lambda x: x.fillna(x.median())
)

# 3. Fill missing email based on name
def generate_email(row):
    if pd.isnull(row['email']):
        first = row['first_name'].lower()
        last = row['last_name'].lower()
        return f"{first}.{last}@company.com"
    return row['email']

employees['email'] = employees.apply(generate_email, axis=1)

# 4. Handle missing bonus (fill with 0, as no bonus)
employees['bonus'] = employees['bonus'].fillna(0)

print("\nAfter cleaning:")
print(employees.isnull().sum())

---
# Section 4: Outlier Detection and Treatment
---

Outliers are extreme values that differ significantly from other observations.

In [ ]:
# Example: Outlier detection methods

employees = pd.read_csv('assets/datasets/employees.csv')

def detect_outliers_iqr(series, multiplier=1.5):
    """Detect outliers using IQR method."""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - multiplier * IQR
    upper = Q3 + multiplier * IQR
    return (series < lower) | (series > upper), lower, upper

def detect_outliers_zscore(series, threshold=3):
    """Detect outliers using Z-score method."""
    z_scores = np.abs((series - series.mean()) / series.std())
    return z_scores > threshold

# Apply to salary
is_outlier_iqr, lower, upper = detect_outliers_iqr(employees['salary'])
is_outlier_zscore = detect_outliers_zscore(employees['salary'])

print("Salary Outlier Detection:")
print(f"IQR Method: {is_outlier_iqr.sum()} outliers (bounds: ${lower:,.0f} - ${upper:,.0f})")
print(f"Z-Score Method: {is_outlier_zscore.sum()} outliers")

In [ ]:
# Example: Visualizing outliers

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Box plot
sns.boxplot(x=employees['salary'], ax=axes[0])
axes[0].set_title('Box Plot - Salary')

# Histogram with bounds
sns.histplot(employees['salary'], ax=axes[1], bins=30)
axes[1].axvline(lower, color='red', linestyle='--', label=f'Lower: ${lower:,.0f}')
axes[1].axvline(upper, color='red', linestyle='--', label=f'Upper: ${upper:,.0f}')
axes[1].legend()
axes[1].set_title('Distribution with IQR Bounds')

# Scatter plot highlighting outliers
colors = ['red' if x else 'blue' for x in is_outlier_iqr]
axes[2].scatter(range(len(employees)), employees['salary'], c=colors, alpha=0.5)
axes[2].set_title('Outliers Highlighted (red)')
axes[2].set_xlabel('Index')
axes[2].set_ylabel('Salary')

plt.tight_layout()
plt.show()

In [ ]:
# Example: Outlier treatment strategies

df = employees.copy()

# Strategy 1: Remove outliers
is_outlier, lower, upper = detect_outliers_iqr(df['salary'])
df_removed = df[~is_outlier]
print(f"Strategy 1 - Remove: {len(df)} -> {len(df_removed)} rows")

# Strategy 2: Cap/Winsorize (clip to bounds)
df['salary_capped'] = df['salary'].clip(lower=lower, upper=upper)
print(f"\nStrategy 2 - Cap:")
print(f"  Original max: ${df['salary'].max():,.0f}")
print(f"  Capped max: ${df['salary_capped'].max():,.0f}")

# Strategy 3: Replace with median
median_salary = df['salary'].median()
df['salary_median'] = df['salary'].copy()
df.loc[is_outlier, 'salary_median'] = median_salary
print(f"\nStrategy 3 - Replace with median (${median_salary:,.0f}): {is_outlier.sum()} values replaced")

In [ ]:
# Example: Compare distributions after treatment

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sns.histplot(df['salary'], ax=axes[0], kde=True)
axes[0].set_title('Original')

sns.histplot(df['salary_capped'], ax=axes[1], kde=True)
axes[1].set_title('Capped (Winsorized)')

sns.histplot(df['salary_median'], ax=axes[2], kde=True)
axes[2].set_title('Outliers Replaced with Median')

plt.tight_layout()
plt.show()

---
# Section 5: Data Normalization and Standardization
---

Scaling numeric features is essential for many machine learning algorithms.

## Scaling Methods

| Method | Formula | Range | Use Case |
|--------|---------|-------|----------|
| **Min-Max** | (x - min) / (max - min) | [0, 1] | Neural networks, bounded data |
| **Standard (Z-score)** | (x - mean) / std | ~[-3, 3] | Most ML algorithms |
| **Robust** | (x - median) / IQR | Varies | Data with outliers |

In [ ]:
# Example: Manual scaling

employees = pd.read_csv('assets/datasets/employees.csv')

# Min-Max scaling
salary = employees['salary']
salary_minmax = (salary - salary.min()) / (salary.max() - salary.min())

# Standard scaling (Z-score)
salary_standard = (salary - salary.mean()) / salary.std()

# Robust scaling
median = salary.median()
iqr = salary.quantile(0.75) - salary.quantile(0.25)
salary_robust = (salary - median) / iqr

# Compare
comparison = pd.DataFrame({
    'Original': salary,
    'MinMax': salary_minmax,
    'Standard': salary_standard,
    'Robust': salary_robust
}).describe()

print("Scaling Comparison:")
print(comparison.round(3))

In [ ]:
# Example: Using scikit-learn scalers

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Prepare data
salary_values = employees[['salary']].values

# MinMax Scaler
minmax_scaler = MinMaxScaler()
salary_minmax = minmax_scaler.fit_transform(salary_values)

# Standard Scaler
standard_scaler = StandardScaler()
salary_standard = standard_scaler.fit_transform(salary_values)

# Robust Scaler
robust_scaler = RobustScaler()
salary_robust = robust_scaler.fit_transform(salary_values)

print("Sklearn Scalers:")
print(f"MinMax - Range: [{salary_minmax.min():.3f}, {salary_minmax.max():.3f}]")
print(f"Standard - Mean: {salary_standard.mean():.3f}, Std: {salary_standard.std():.3f}")
print(f"Robust - Median: {np.median(salary_robust):.3f}")

In [ ]:
# Example: Visualize scaling effects

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Original
axes[0, 0].hist(salary_values, bins=30, edgecolor='white')
axes[0, 0].set_title('Original')
axes[0, 0].set_xlabel('Salary')

# MinMax
axes[0, 1].hist(salary_minmax, bins=30, edgecolor='white', color='green')
axes[0, 1].set_title('Min-Max Scaled [0, 1]')

# Standard
axes[1, 0].hist(salary_standard, bins=30, edgecolor='white', color='orange')
axes[1, 0].set_title('Standard Scaled (Z-score)')

# Robust
axes[1, 1].hist(salary_robust, bins=30, edgecolor='white', color='purple')
axes[1, 1].set_title('Robust Scaled')

plt.tight_layout()
plt.show()

In [ ]:
# Example: Scaling multiple columns

# Prepare numeric data
employees = pd.read_csv('assets/datasets/employees.csv')
employees['hire_date'] = pd.to_datetime(employees['hire_date'])
employees['years_exp'] = (pd.Timestamp.now() - employees['hire_date']).dt.days / 365
employees['bonus'] = employees['bonus'].fillna(0)

# Select numeric columns to scale
numeric_cols = ['salary', 'bonus', 'years_exp']
numeric_data = employees[numeric_cols].copy()

# Apply StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_data)
scaled_df = pd.DataFrame(scaled_data, columns=[f"{col}_scaled" for col in numeric_cols])

print("Scaled Data Statistics:")
print(scaled_df.describe().round(3))

---
# Section 6: Encoding Categorical Variables
---

Machine learning algorithms require numeric input. Categorical variables must be encoded.

In [ ]:
# Example: Label Encoding (ordinal)

from sklearn.preprocessing import LabelEncoder

employees = pd.read_csv('assets/datasets/employees.csv')

# Label encode status (has inherent order)
le = LabelEncoder()
employees['status_encoded'] = le.fit_transform(employees['status'])

print("Label Encoding - Status:")
print(pd.DataFrame({'Original': le.classes_, 'Encoded': range(len(le.classes_))}))

In [ ]:
# Example: One-Hot Encoding (nominal)

# Using pandas get_dummies
dept_encoded = pd.get_dummies(employees['department'], prefix='dept')
print("One-Hot Encoding - Department:")
print(dept_encoded.head())

In [ ]:
# Example: One-Hot Encoding with sklearn

from sklearn.preprocessing import OneHotEncoder

# Prepare data
dept_values = employees[['department']]

# Fit and transform
ohe = OneHotEncoder(sparse_output=False, drop='first')  # drop first to avoid multicollinearity
dept_encoded = ohe.fit_transform(dept_values)

# Create DataFrame
dept_df = pd.DataFrame(
    dept_encoded, 
    columns=ohe.get_feature_names_out(['department'])
)

print("One-Hot Encoding (sklearn, drop first):")
print(dept_df.head())
print(f"\nShape: {dept_df.shape}")

In [ ]:
# Example: Ordinal Encoding (for ordered categories)

from sklearn.preprocessing import OrdinalEncoder

# Create sample data with ordered categories
education_data = pd.DataFrame({
    'education': ['High School', 'Bachelor', 'Master', 'PhD', 'Bachelor', 
                  'High School', 'Master', 'Bachelor', 'PhD', 'High School']
})

# Define order
education_order = [['High School', 'Bachelor', 'Master', 'PhD']]

# Encode
oe = OrdinalEncoder(categories=education_order)
education_data['education_encoded'] = oe.fit_transform(education_data[['education']])

print("Ordinal Encoding - Education:")
print(education_data.drop_duplicates().sort_values('education_encoded'))

In [ ]:
# Example: Target/Mean Encoding (for high-cardinality)

sales = pd.read_csv('assets/datasets/sales_data.csv')

# Calculate mean target for each product
product_means = sales.groupby('product')['total_amount'].mean()

# Map to original data
sales['product_encoded'] = sales['product'].map(product_means)

print("Target/Mean Encoding - Product:")
print(sales[['product', 'product_encoded']].drop_duplicates().head(10))

## Practice Exercise 6.1

**Task:** Prepare the employees dataset for machine learning:
1. One-hot encode the department column
2. Ordinal encode the status column (Active=2, On Leave=1, Terminated=0)
3. Standard scale the salary and years of experience

In [ ]:
# Your code here


In [ ]:
# Solution 6.1

from sklearn.preprocessing import StandardScaler, OrdinalEncoder

employees = pd.read_csv('assets/datasets/employees.csv')
employees['hire_date'] = pd.to_datetime(employees['hire_date'])
employees['years_exp'] = (pd.Timestamp.now() - employees['hire_date']).dt.days / 365

# 1. One-hot encode department
dept_dummies = pd.get_dummies(employees['department'], prefix='dept', drop_first=True)
print("1. Department One-Hot Encoded:")
print(dept_dummies.columns.tolist())

# 2. Ordinal encode status
status_order = [['Terminated', 'On Leave', 'Active']]
oe = OrdinalEncoder(categories=status_order)
employees['status_encoded'] = oe.fit_transform(employees[['status']])
print("\n2. Status Ordinal Encoded:")
print(employees[['status', 'status_encoded']].drop_duplicates())

# 3. Standard scale salary and years_exp
scaler = StandardScaler()
employees[['salary_scaled', 'years_exp_scaled']] = scaler.fit_transform(
    employees[['salary', 'years_exp']]
)
print("\n3. Scaled Variables:")
print(employees[['salary', 'salary_scaled', 'years_exp', 'years_exp_scaled']].describe().round(3))

---
# Section 7: Feature Engineering Basics
---

Feature engineering creates new features from existing data to improve model performance.

In [ ]:
# Example: DateTime features

sales = pd.read_csv('assets/datasets/sales_data.csv', parse_dates=['date'])

# Extract datetime components
sales['year'] = sales['date'].dt.year
sales['month'] = sales['date'].dt.month
sales['day'] = sales['date'].dt.day
sales['day_of_week'] = sales['date'].dt.dayofweek
sales['day_name'] = sales['date'].dt.day_name()
sales['quarter'] = sales['date'].dt.quarter
sales['is_weekend'] = sales['day_of_week'].isin([5, 6]).astype(int)
sales['is_month_end'] = sales['date'].dt.is_month_end.astype(int)

print("DateTime Features:")
print(sales[['date', 'year', 'month', 'day_of_week', 'is_weekend', 'quarter']].head())

In [ ]:
# Example: Mathematical transformations

sales = pd.read_csv('assets/datasets/sales_data.csv')

# Log transformation (for skewed data)
sales['log_amount'] = np.log1p(sales['total_amount'])  # log1p handles zeros

# Square root transformation
sales['sqrt_amount'] = np.sqrt(sales['total_amount'])

# Polynomial features
sales['amount_squared'] = sales['total_amount'] ** 2

# Compare distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(sales['total_amount'], bins=30)
axes[0].set_title('Original (Skewed)')

axes[1].hist(sales['log_amount'], bins=30, color='green')
axes[1].set_title('Log Transformed')

axes[2].hist(sales['sqrt_amount'], bins=30, color='orange')
axes[2].set_title('Square Root Transformed')

plt.tight_layout()
plt.show()

In [ ]:
# Example: Binning/Discretization

employees = pd.read_csv('assets/datasets/employees.csv')

# Equal-width bins
employees['salary_bin'] = pd.cut(employees['salary'], bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

# Quantile-based bins (equal frequency)
employees['salary_quantile'] = pd.qcut(employees['salary'], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])

# Custom bins
bins = [0, 50000, 75000, 100000, 150000, float('inf')]
labels = ['Entry', 'Junior', 'Mid', 'Senior', 'Executive']
employees['salary_level'] = pd.cut(employees['salary'], bins=bins, labels=labels)

print("Binned Salary:")
print(employees[['salary', 'salary_bin', 'salary_quantile', 'salary_level']].head(10))

In [ ]:
# Example: Aggregation features

sales = pd.read_csv('assets/datasets/sales_data.csv')

# Product-level statistics
product_stats = sales.groupby('product').agg(
    avg_amount=('total_amount', 'mean'),
    total_sales=('total_amount', 'sum'),
    transaction_count=('transaction_id', 'count')
).reset_index()

# Merge back to original data
sales = sales.merge(product_stats, on='product', suffixes=('', '_product'))

# Calculate deviation from product average
sales['amount_vs_product_avg'] = sales['total_amount'] - sales['avg_amount']

print("Aggregation Features:")
print(sales[['product', 'total_amount', 'avg_amount', 'amount_vs_product_avg']].head())

In [ ]:
# Example: Text-based features

employees = pd.read_csv('assets/datasets/employees.csv')

# Name length
employees['name_length'] = (employees['first_name'] + ' ' + employees['last_name']).str.len()

# First letter
employees['first_letter'] = employees['first_name'].str[0]

# Title word count
employees['title_words'] = employees['title'].str.split().str.len()

# Contains 'Senior' or 'Manager'
employees['is_senior'] = employees['title'].str.contains('Senior|Lead', case=False).astype(int)
employees['is_manager'] = employees['title'].str.contains('Manager', case=False).astype(int)

print("Text-based Features:")
print(employees[['first_name', 'title', 'is_senior', 'is_manager', 'title_words']].head(10))

In [ ]:
# Example: Interaction features

employees = pd.read_csv('assets/datasets/employees.csv')
employees['hire_date'] = pd.to_datetime(employees['hire_date'])
employees['years_exp'] = (pd.Timestamp.now() - employees['hire_date']).dt.days / 365
employees['bonus'] = employees['bonus'].fillna(0)
employees['performance_rating'] = employees['performance_rating'].fillna(3)

# Salary per year of experience
employees['salary_per_year'] = employees['salary'] / (employees['years_exp'] + 1)

# Total compensation
employees['total_compensation'] = employees['salary'] + employees['bonus']

# Bonus ratio
employees['bonus_ratio'] = employees['bonus'] / employees['salary']

# Performance x Experience interaction
employees['perf_exp_interaction'] = employees['performance_rating'] * employees['years_exp']

print("Interaction Features:")
print(employees[['salary', 'bonus', 'years_exp', 'total_compensation', 'salary_per_year']].head())

---
# Section 8: Data Cleaning Pipeline Project
---

Let's build a complete, reusable data cleaning pipeline.

In [ ]:
# Example: Data Cleaning Pipeline Class

class DataCleaningPipeline:
    """A reusable data cleaning pipeline."""
    
    def __init__(self):
        self.cleaning_log = []
        self.original_shape = None
        
    def log(self, message):
        """Add message to cleaning log."""
        self.cleaning_log.append(message)
        print(f"  -> {message}")
    
    def fit_transform(self, df):
        """Apply all cleaning steps."""
        print("\n" + "="*60)
        print("DATA CLEANING PIPELINE")
        print("="*60)
        
        self.original_shape = df.shape
        df = df.copy()
        
        # Step 1: Remove duplicates
        print("\nStep 1: Removing Duplicates")
        initial_rows = len(df)
        df = df.drop_duplicates()
        self.log(f"Removed {initial_rows - len(df)} duplicate rows")
        
        # Step 2: Handle missing values
        print("\nStep 2: Handling Missing Values")
        for col in df.columns:
            missing = df[col].isnull().sum()
            if missing > 0:
                if df[col].dtype in ['float64', 'int64']:
                    median_val = df[col].median()
                    df[col] = df[col].fillna(median_val)
                    self.log(f"{col}: filled {missing} missing with median ({median_val:.2f})")
                else:
                    mode_val = df[col].mode()[0] if len(df[col].mode()) > 0 else 'Unknown'
                    df[col] = df[col].fillna(mode_val)
                    self.log(f"{col}: filled {missing} missing with mode ({mode_val})")
        
        # Step 3: Fix data types
        print("\nStep 3: Fixing Data Types")
        for col in df.columns:
            if 'date' in col.lower():
                try:
                    df[col] = pd.to_datetime(df[col])
                    self.log(f"{col}: converted to datetime")
                except:
                    pass
        
        # Step 4: Standardize text
        print("\nStep 4: Standardizing Text")
        for col in df.select_dtypes(include='object').columns:
            df[col] = df[col].str.strip()
            self.log(f"{col}: stripped whitespace")
        
        print("\n" + "="*60)
        print(f"COMPLETE: {self.original_shape} -> {df.shape}")
        print("="*60)
        
        return df
    
    def get_report(self):
        """Return cleaning report."""
        return "\n".join(self.cleaning_log)

In [ ]:
# Example: Using the pipeline

# Load data
sales = pd.read_csv('assets/datasets/sales_data.csv')

# Create and run pipeline
pipeline = DataCleaningPipeline()
clean_sales = pipeline.fit_transform(sales)

In [ ]:
# Example: Complete preprocessing function

def preprocess_employees(df):
    """
    Complete preprocessing pipeline for employees dataset.
    Returns cleaned and feature-engineered DataFrame.
    """
    df = df.copy()
    
    # 1. Handle dates
    df['hire_date'] = pd.to_datetime(df['hire_date'])
    df['years_exp'] = (pd.Timestamp.now() - df['hire_date']).dt.days / 365
    
    # 2. Handle missing values
    df['performance_rating'] = df.groupby('department')['performance_rating'].transform(
        lambda x: x.fillna(x.median())
    )
    df['bonus'] = df['bonus'].fillna(0)
    df['email'] = df.apply(
        lambda row: f"{row['first_name'].lower()}.{row['last_name'].lower()}@company.com" 
        if pd.isnull(row['email']) else row['email'], axis=1
    )
    
    # 3. Feature engineering
    df['total_compensation'] = df['salary'] + df['bonus']
    df['salary_per_year'] = df['salary'] / (df['years_exp'] + 1)
    df['is_manager'] = df['title'].str.contains('Manager', case=False).astype(int)
    df['is_senior'] = df['title'].str.contains('Senior|Lead', case=False).astype(int)
    
    # 4. Encode categoricals
    df['status_encoded'] = df['status'].map({'Terminated': 0, 'On Leave': 1, 'Active': 2})
    
    # 5. Remove unnecessary columns
    df = df.drop(['hire_date'], axis=1)
    
    return df

# Apply preprocessing
employees = pd.read_csv('assets/datasets/employees.csv')
employees_clean = preprocess_employees(employees)

print("Preprocessed Employees:")
print(employees_clean.head())
print(f"\nShape: {employees_clean.shape}")
print(f"\nColumns: {employees_clean.columns.tolist()}")

## Practice Exercise 8.1

**Task:** Create a complete preprocessing pipeline for the sales data that:
1. Handles missing values
2. Creates datetime features (year, month, day_of_week, is_weekend)
3. Creates aggregation features (product average, region average)
4. One-hot encodes category and region
5. Returns a clean DataFrame ready for analysis

In [ ]:
# Your code here


In [ ]:
# Solution 8.1

def preprocess_sales(df):
    """
    Complete preprocessing pipeline for sales dataset.
    """
    df = df.copy()
    
    # 1. Handle missing values
    df['unit_price'] = df.groupby('category')['unit_price'].transform(
        lambda x: x.fillna(x.median())
    )
    df['sales_rep'] = df['sales_rep'].fillna('Unknown')
    df['customer_rating'] = df['customer_rating'].fillna(df['customer_rating'].median())
    
    # 2. DateTime features
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day_of_week'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['quarter'] = df['date'].dt.quarter
    
    # 3. Aggregation features
    product_avg = df.groupby('product')['total_amount'].transform('mean')
    df['product_avg_amount'] = product_avg
    df['vs_product_avg'] = df['total_amount'] - product_avg
    
    region_avg = df.groupby('region')['total_amount'].transform('mean')
    df['region_avg_amount'] = region_avg
    
    # 4. One-hot encode
    category_dummies = pd.get_dummies(df['category'], prefix='cat', drop_first=True)
    region_dummies = pd.get_dummies(df['region'], prefix='reg', drop_first=True)
    df = pd.concat([df, category_dummies, region_dummies], axis=1)
    
    # 5. Clean up
    df = df.drop(['date'], axis=1)
    
    print(f"Preprocessing complete: {df.shape[0]} rows, {df.shape[1]} columns")
    print(f"Missing values remaining: {df.isnull().sum().sum()}")
    
    return df

# Apply
sales = pd.read_csv('assets/datasets/sales_data.csv')
sales_clean = preprocess_sales(sales)

print("\nNew columns:")
new_cols = [c for c in sales_clean.columns if c not in sales.columns]
print(new_cols)

---
# Module Summary

## Key Takeaways

1. **Data quality is critical**: Garbage in, garbage out
2. **Duplicates**: Identify and remove based on business logic
3. **Missing values**: Choose strategy based on pattern and importance
4. **Outliers**: Detect with IQR/Z-score, treat appropriately
5. **Scaling**: Essential for many ML algorithms
6. **Encoding**: Convert categories to numbers for ML
7. **Feature engineering**: Create informative features from raw data
8. **Pipelines**: Make preprocessing reproducible and reusable

## Common Preprocessing Steps

```python
# Typical workflow
1. df.drop_duplicates()           # Remove duplicates
2. df.fillna() / df.dropna()      # Handle missing
3. Outlier detection/treatment     # Handle outliers
4. pd.to_datetime()               # Fix date types
5. StandardScaler / MinMaxScaler  # Scale numeric
6. pd.get_dummies() / LabelEncoder # Encode categorical
7. Feature engineering             # Create new features
```

## Next Module

In the next module, we'll cover **Statistics for Data Science** - the statistical foundations needed for data analysis and machine learning.

## Additional Practice

1. **End-to-End Pipeline**: Create a complete preprocessing pipeline that can be saved and loaded for new data.

2. **Missing Value Analysis**: Compare different imputation strategies on a dataset and evaluate their impact on model performance.

3. **Feature Engineering Challenge**: Create at least 10 new features from the sales data that might be predictive of transaction amount.